# Benchmark Trained Gemma Models vs Off the Shelf Gemma on Test Set with win rate

This notebook benchmarks the trained Gemma models compared with an Off the Shelf Gemma using winrate on the test set

## 1. Install Dependencies

First, let's make sure we have all required packages.

In [1]:
from unsloth import FastModel
import torch
import torch.nn as nn
from datasets import load_dataset
import re
from trl import GRPOConfig, GRPOTrainer
from transformers import (
    GPT2Model,
    GPT2Tokenizer,
    GPT2PreTrainedModel,
    GPT2Config,
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM,
    TextStreamer,
 AutoTokenizer
)
from typing import Dict, List
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
from datasets import Dataset as HFDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import datetime
import time
from sklearn.preprocessing import StandardScaler
import pickle
import math

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-07 09:01:02 [__init__.py:256] Automatically detected platform cuda.


In [2]:
# Set seed
seed_value = 46
torch.manual_seed(seed_value)

## 3. Configure Benchmark Parameters

Set the parameters for your benchmark run.

In [3]:
# --- Configuration ---
# Ensure this path points to where your fine-tuned model was saved
# It should contain 'adapter_config.json', 'adapter_model.safetensors', etc.
MODEL_PATH = "gemma_glicko_pess" 
# The base model used for fine-tuning
BASE_MODEL = "unsloth/gemma-3-1b-it" 
MAX_SEQ_LENGTH = 600
DATASET_NAME = "Columbia-NLP/DPO-tldr-summarisation-preferences"
NUM_SAMPLES_TO_GENERATE = 10 # Adjust as needed, use -1 for the whole test set
OUTPUT_CSV = "generated_summaries_gemma_grpo.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# # --- Load Model and Tokenizer ---
# model_base, tokenizer_base = FastModel.from_pretrained(
#         model_name = "gemma_glicko_base", # Load the adapter
#         max_seq_length = MAX_SEQ_LENGTH,
#         load_in_4bit = False,
#         load_in_8bit = False,
#     )
# model_pess, tokenizer_pess = FastModel.from_pretrained(
#         model_name = "gemma_glicko_pess", # Load the adapter
#         max_seq_length = MAX_SEQ_LENGTH,
#         load_in_4bit = False,
#         load_in_8bit = False,
#     )

In [5]:
# # --- Setup Device and Tokenizer ---
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_base.to(device)
# model_base.eval() # Set model to evaluation mode
# model_pess.to(device)
# model_pess.eval() # Set model to evaluation mode

In [6]:
# --- Load Dataset ---
test_sr = ['running','Cooking', 'books', 'jobs', 'cats', 'travel', 'Pets', 'dogs', 'offmychest', 'self', 'college', 'personalfinance']
print(f"Loading dataset: {DATASET_NAME}...")
dataset = load_dataset(DATASET_NAME)
test_set = dataset['test']
dataset = test_set.add_column("sub_reddit", [x['subreddit'] for x in test_set['other_info']])
df = dataset.to_pandas()
test_df = df.loc[ df['sub_reddit'].isin(test_sr)]
dataset = HFDataset.from_pandas(test_df)
print("Dataset loaded.")

Loading dataset: Columbia-NLP/DPO-tldr-summarisation-preferences...
Dataset loaded.


In [7]:
test_df.head()

,prompt,prompt_id,chosen,rejected,messages,score_chosen,score_rejected,other_info,sub_reddit
29,You are an AI assistant good at summarizing re...,17044c46e73997247c4780d0784be3ddaeca39552f81fe...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,10.0,1.0,"{'chosen_note': 'clear. ', 'id': 't3_1g...",dogs
30,You are an AI assistant good at summarizing re...,17044c46e73997247c4780d0784be3ddaeca39552f81fe...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,10.0,1.0,"{'chosen_note': 'clear. ', 'id': 't3_1gyf...",dogs
31,You are an AI assistant good at summarizing re...,17044c46e73997247c4780d0784be3ddaeca39552f81fe...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,10.0,1.0,"{'chosen_note': 'clear. ', 'id': ...",dogs
32,You are an AI assistant good at summarizing re...,17044c46e73997247c4780d0784be3ddaeca39552f81fe...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,10.0,1.0,"{'chosen_note': 'clear. ', 'id': 't3_1g...",dogs
33,You are an AI assistant good at summarizing re...,17044c46e73997247c4780d0784be3ddaeca39552f81fe...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,[{'content': 'You are an AI assistant good at ...,10.0,1.0,"{'chosen_note': 'clear.', 'id': 't3_1gyf5t', '...",dogs


In [8]:
def generate_summaries(prompt_texts, model, tokenizer, max_new_tokens=53):
    """
    Generates summaries for a batch of prompts using the loaded model.
    Args:
        prompt_texts (list of str): A list of prompts to summarize.
        model: The loaded Hugging Face model.
        tokenizer: The loaded Hugging Face tokenizer.
        max_new_tokens (int): Maximum number of new tokens to generate for each summary.
    Returns:
        list of str: A list of generated summaries.
    """
    inputs = tokenizer(
        prompt_texts, # Process a list of prompts
        return_tensors="pt",
        padding=True, # Pad to the longest sequence in the batch
        padding_side = 'left',
        truncation=True,
        max_length=MAX_SEQ_LENGTH - max_new_tokens # Make space for generated text
    ).to(device)

    with torch.no_grad(): # Disable gradient calculation for inference
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,      # Use sampling
            temperature=0.1,     # Lower temperature for less randomness
            top_p=1,           # Nucleus sampling
            num_return_sequences=1,
            min_length = 53,
        )

    generated_summaries = []
    # Decode each summary in the batch
    # outputs contains the full sequence (prompt + summary)
    # We need to slice off the prompt part for each generated summary
    input_ids_length = inputs.input_ids.shape[1] # Length of the tokenized input prompts (padded)
    for i in range(outputs.shape[0]): # Iterate through each item in the batch
        summary_ids = outputs[i][input_ids_length:]
        summary = tokenizer.decode(summary_ids, skip_special_tokens=True)
        generated_summaries.append(summary.strip())
    return generated_summaries

    
def responses(path, prompts): 
    model, tokenizer = FastModel.from_pretrained(
        model_name = path, # Load the adapter
        max_seq_length = MAX_SEQ_LENGTH,
        load_in_4bit = False,
        load_in_8bit = False,
    )
    model.to(device)
    model.eval() 
    answers = []
    num = 100
    for i in range(math.ceil(len(prompts)/num)):
        answers+=generate_summaries(prompts[i*num:(i+1)*num], model, tokenizer)
    return answers
    #  answers = []
    # num = 10
    # l = len(prompts)//num
    # for i in range(num):
    #     answers += generate_summaries(prompts[i*l:(i+1*l)], model, tokenizer)
    # return answers

In [9]:
# Sample Entries for the test set
n = 500 
test = test_df.sample(n)
test = test[['prompt', 'chosen']]
test['prompt'] = [t[:len(t)-8] + 'Summarize the post in two sentences:' for t in test['prompt']]
test['chosen'] = [x[1]['content'] for x in test['chosen']]
test.reset_index()

,index,prompt,chosen
0,33673,You are an AI assistant good at summarizing re...,"Currently a graphic design major, not as passi..."
1,16321,You are an AI assistant good at summarizing re...,fiancé accidentally signed up for a credit car...
2,45601,You are an AI assistant good at summarizing re...,I've been able to read conversations going bac...
3,22159,You are an AI assistant good at summarizing re...,Need a family lawyer's advice...
4,34224,You are an AI assistant good at summarizing re...,My wife is driving like a maniac and I'm not s...
...,...,...,...
495,32074,You are an AI assistant good at summarizing re...,"Company got bought out, new contract is insane..."
496,18270,You are an AI assistant good at summarizing re...,my neighbors are too loud all the time and it'...
497,23555,You are an AI assistant good at summarizing re...,my cousin's parental rights were removed for d...
498,13555,You are an AI assistant good at summarizing re...,Bought a puppy from a puppy mill. I feel stupi...


In [10]:
# test = test_df[['prompt', 'chosen']]
# test['prompt'] = [t[:len(t)-8] + 'Summarize the post in two sentences:' for t in test['prompt']]
# test['chosen'] = [x[1]['content'] for x in test['chosen']]
# test.drop_duplicates('prompt', inplace=True)
# test.reset_index()

In [11]:
prompts = test['prompt'].to_list()

In [12]:
import time 
import warnings
warnings.filterwarnings('ignore') # To ignore all warnings
start = time.time()
pess = responses("gemma_glicko_pess", prompts)
base = responses("gemma_glicko_base", prompts)
print(time.time() - start)

==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.999 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.
==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.999 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - i

In [13]:
len('.\n\n**Response:**\n\n')

18

In [14]:
t = []
u = []
for b in base:
    i = b.rfind('**')
    if i != -1:
        t.append(b[i+2:].replace("\n", ""))
    else:
        t.append(b.replace("\n", ""))
for b in pess:
    i = b.rfind('**')
    if i != -1:
        u.append(b[i+2:].replace("\n", ""))
    else:
        u.append(b.replace("\n", ""))

In [15]:
for a in pess[:10]:
    print(a)
    print('____________')

The author is a sophomore graphic design major who initially had a strong interest in graphic design but has now realized it's not a fulfilling career path. They are hesitant to switch majors but are exploring a plan to broaden their education by taking general education courses to explore
____________
The fiancé signed up for a points card at the commissary, a credit card, and the signup was a mistake.  He initially believed it was a points card, but the credit card was revealed, leading to a potential credit card issue and a need to dispute
____________
The author is concerned about a potential relationship with their father, and has been reading his iMessage conversations, leading to a sense of unease and a fear of being monitored.  The author feels a lack of trust and a strained relationship with their parents, making
____________
The author is a 15-year-old sibling with a strained relationship with their parents and a deeply fractured family dynamic. They are seeking a custody arr

In [16]:
human = test['chosen'].to_list()

In [17]:
prompts = [x[:len(x)-38] for x in prompts]

In [18]:
df = pd.DataFrame({'Prompt' : prompts, 'Base':base, 'Pessimism' : pess, 'Human' : human}) 

In [19]:
df.to_csv('summaries' + str(seed_value) +'.csv', index=False)